# Generative Adversarial Network

In this notebook, we'll be building a generative adversarial network (GAN) trained on the network flow dataset.

## Import Libraries

In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload
import os
import time
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow version: 1.8.0
Eager execution: True


In [2]:
import utils, models
reload(utils)
reload(models)
from models import Generator, Discriminator
from utils import max_norm, parse_feature_label, train_one_epoch

## Define Constants

In [3]:
IDS_DATASET = os.path.join('data', 'ids2017_sampled.csv')
RELEVANT_FEATURES = [' Source Port', ' Destination Port', ' Flow Duration', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', 'Bwd Packet Length Max', ' Bwd Packet Length Min', 'Flow Bytes/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Packets/s', ' Packet Length Mean', ' ACK Flag Count', ' Down/Up Ratio', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Bwd Avg Bytes/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' act_data_pkt_fwd', ' Active Std', ' Active Min', ' Idle Max']
LABEL_NAME = ' Label'
BENIGN_LABEL = 0
ATTACK_LABEL = 2
TRAIN_FRAC = 0.3
FEATURE_NUM_MODIFIED = 2
LEARNING_RATE = 0.01
OUTPUT_DIR = 'SUMMARY/'
CHECKPOINT_DIR = 'CHECKPOINT/'
EPOCHS = 20000
LOG_INTERVAL = 100

## Load and normalize data

In [4]:
# quick view of dataset
!head -n5 {IDS_DATASET}

Flow ID, Source IP, Source Port, Destination IP, Destination Port, Protocol, Timestamp, Flow Duration, Total Fwd Packets, Total Backward Packets,Total Length of Fwd Packets, Total Length of Bwd Packets, Fwd Packet Length Max, Fwd Packet Length Min, Fwd Packet Length Mean, Fwd Packet Length Std,Bwd Packet Length Max, Bwd Packet Length Min, Bwd Packet Length Mean, Bwd Packet Length Std,Flow Bytes/s, Flow Packets/s, Flow IAT Mean, Flow IAT Std, Flow IAT Max, Flow IAT Min,Fwd IAT Total, Fwd IAT Mean, Fwd IAT Std, Fwd IAT Max, Fwd IAT Min,Bwd IAT Total, Bwd IAT Mean, Bwd IAT Std, Bwd IAT Max, Bwd IAT Min,Fwd PSH Flags, Bwd PSH Flags, Fwd URG Flags, Bwd URG Flags, Fwd Header Length, Bwd Header Length,Fwd Packets/s, Bwd Packets/s, Min Packet Length, Max Packet Length, Packet Length Mean, Packet Length Std, Packet Length Variance,FIN Flag Count, SYN Flag Count, RST Flag Count, PSH Flag Count, ACK Flag Count, URG Flag Count, CWE Flag Count, ECE Flag Count, Down/Up Ratio, Average Packet Size, Av

In [5]:
# read csv
df = pd.read_csv(IDS_DATASET)

In [6]:
# extract relevant features and label name
df = df[RELEVANT_FEATURES + [LABEL_NAME]]

In [7]:
# extract bengin and attack flows we want
benign_df, attack_df = df[(df[LABEL_NAME] == BENIGN_LABEL)], df[(df[LABEL_NAME] == ATTACK_LABEL)]

In [8]:
# rewrite label values
benign_df.loc[:, LABEL_NAME] = 0
attack_df.loc[:, LABEL_NAME] = 1

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
# convert to numpy
benign, attack = benign_df.values, attack_df.values

In [10]:
# max normalization
benign[:, :len(RELEVANT_FEATURES)] = max_norm(benign[:, :len(RELEVANT_FEATURES)])
attack[:, :len(RELEVANT_FEATURES)] = max_norm(attack[:, :len(RELEVANT_FEATURES)])

In [11]:
# do train, test split separately on benign and attack
benign_train, benign_test = train_test_split(benign, train_size=TRAIN_FRAC)
attack_train, attack_test = train_test_split(attack, train_size=TRAIN_FRAC)

/scratch/sj2363/hsn/attack_generate/gan_attack_generate/.env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [12]:
# concat to get testing data
test_np = np.concatenate([benign_test, attack_test])

## Generate Dataset

In [13]:
# convert to benign train dataset and attack train dataset
benign_train_dataset, attack_train_dataset = tf.data.Dataset.from_tensor_slices(benign_train), tf.data.Dataset.from_tensor_slices(attack_train)

In [14]:
# deal with benign train dataset
benign_train_dataset = benign_train_dataset.map(parse_feature_label)
benign_train_dataset = benign_train_dataset.shuffle(buffer_size=benign_train.shape[0] * 5)  # randomize
benign_train_dataset = benign_train_dataset.batch(100) # make batch

# View a single example entry from a batch
benign_features, benign_label = iter(benign_train_dataset).next()
print("benign example features:", benign_features[0])
print("benign example label:", benign_label[0])

benign example features: tf.Tensor(
[5.49082751e-01 1.24760226e-03 9.63148282e-01 5.26492221e-03
 2.13725490e-04 2.47945205e-01 0.00000000e+00 1.19523015e-02
 4.62273032e-02 6.64731022e-02 8.47457705e-02 4.68018713e-08
 9.66666667e-01 5.44084736e-02 7.15016118e-02 1.18644068e-06
 9.25000000e-01 4.68762157e-02 8.05734510e-02 4.15254237e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.63807133e-03
 1.81718122e-07 2.35987602e-01 0.00000000e+00 1.42857143e-01
 1.81793423e-02 1.63807133e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 9.65250965e-04 5.26492221e-03
 4.45571899e-01 1.08742932e-04 2.19576629e-03 4.71864349e-03
 8.47457627e-02], shape=(41,), dtype=float64)
benign example label: tf.Tensor(0, shape=(), dtype=int32)


In [15]:
# deal with attack train dataset
attack_train_dataset = attack_train_dataset.map(parse_feature_label)
attack_train_dataset = attack_train_dataset.shuffle(buffer_size=attack_train.shape[0] * 5)  # randomize
attack_train_dataset = attack_train_dataset.batch(100) # make batch

# View a single example entry from a batch
attack_features, attack_label = iter(attack_train_dataset).next()
print("attack example features:", attack_features[0])
print("attack example label:", attack_label[0])

attack example features: tf.Tensor(
[2.23568594e-01 0.00000000e+00 8.36151590e-01 1.45502646e-01
 1.00000000e+00 4.99525658e-01 0.00000000e+00 1.14195331e-05
 1.81220203e-01 4.52218430e-01 8.41525422e-01 5.51146384e-05
 8.34453780e-01 3.15521620e-01 5.19230769e-01 0.00000000e+00
 1.21345378e-03 2.83139216e-04 1.01356374e-03 8.82352941e-08
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.86538462e-01
 6.03296090e-08 4.97746056e-01 1.00000000e+00 0.00000000e+00
 1.28654971e-01 5.86538462e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.83333333e-01 1.45502646e-01
 8.59589041e-03 2.22222222e-01 0.00000000e+00 6.19322616e-02
 8.41525424e-01], shape=(41,), dtype=float64)
attack example label: tf.Tensor(1, shape=(), dtype=int32)


## Build Model

In [16]:
generator = Generator(input_shape=len(RELEVANT_FEATURES), output_shape=FEATURE_NUM_MODIFIED)
discriminator = Discriminator()

## Build Optimizer

In [17]:
generator_optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)

## Build Tensorflow Training Environment

In [18]:
step_counter = tf.train.get_or_create_global_step()
summary_writer = tf.contrib.summary.create_file_writer(
      OUTPUT_DIR, flush_millis=1000)
checkpoint_prefix = os.path.join(CHECKPOINT_DIR, 'ckpt')
latest_cpkt = tf.train.latest_checkpoint(CHECKPOINT_DIR)
if latest_cpkt:
    print('Using latest checkpoint at ' + latest_cpkt)
model_objects = {
    'generator': generator,
    'discriminator': discriminator,
    'generator_optimizer': generator_optimizer,
    'discriminator_optimizer': discriminator_optimizer,
    'step_counter': step_counter
}
checkpoint = tfe.Checkpoint(**model_objects)
# Restore variables on creation if a checkpoint exists.
checkpoint.restore(latest_cpkt)

## Train

In [ ]:
with tf.device('/gpu:0'):
    train_start = time.time()
    for _ in range(EPOCHS):
        start = time.time()
        with summary_writer.as_default():
            train_one_epoch(benign_dataset=benign_train_dataset,
                        attack_dataset=attack_train_dataset,
                        log_interval=LOG_INTERVAL,
                        modified_feature_num=FEATURE_NUM_MODIFIED,
                        **model_objects)
        end = time.time()
        checkpoint.save(checkpoint_prefix)
        print('\nTrain time for epoch #%d (step %d): %f' %
            (checkpoint.save_counter.numpy(),
             checkpoint.step_counter.numpy(),
             end - start))
    print('\nTotal training time for {epoch} epoch(s) is {second}'.format(
        second=time.time() - train_start,
        epoch=EPOCHS
    ))


Train time for epoch #1 (step 15): 1.555715

Train time for epoch #2 (step 30): 1.446816

Train time for epoch #3 (step 45): 1.250942

Train time for epoch #4 (step 60): 1.322122

Train time for epoch #5 (step 75): 1.267903

Train time for epoch #6 (step 90): 1.280822

Train time for epoch #7 (step 105): 1.217464

Train time for epoch #8 (step 120): 1.322068

Train time for epoch #9 (step 135): 1.273258

Train time for epoch #10 (step 150): 1.368589

Train time for epoch #11 (step 165): 1.258015

Train time for epoch #12 (step 180): 1.303410

Train time for epoch #13 (step 195): 1.383821

Train time for epoch #14 (step 210): 1.286100

Train time for epoch #15 (step 225): 1.271227

Train time for epoch #16 (step 240): 1.468140

Train time for epoch #17 (step 255): 1.277793

Train time for epoch #18 (step 270): 1.165784

Train time for epoch #19 (step 285): 1.264591

Train time for epoch #20 (step 300): 1.246740

Train time for epoch #21 (step 315): 1.333488

Train time for epoch #22 (s


Train time for epoch #172 (step 2580): 1.275427

Train time for epoch #173 (step 2595): 1.410812

Train time for epoch #174 (step 2610): 1.333143

Train time for epoch #175 (step 2625): 1.322443

Train time for epoch #176 (step 2640): 1.294370

Train time for epoch #177 (step 2655): 1.368173

Train time for epoch #178 (step 2670): 1.265798

Train time for epoch #179 (step 2685): 1.392356

Train time for epoch #180 (step 2700): 1.196871

Train time for epoch #181 (step 2715): 1.271660

Train time for epoch #182 (step 2730): 1.264716

Train time for epoch #183 (step 2745): 1.300873

Train time for epoch #184 (step 2760): 1.415657

Train time for epoch #185 (step 2775): 1.260868

Train time for epoch #186 (step 2790): 1.244064

Train time for epoch #187 (step 2805): 1.200758

Train time for epoch #188 (step 2820): 1.411614

Train time for epoch #189 (step 2835): 1.285943

Train time for epoch #190 (step 2850): 1.250840

Train time for epoch #191 (step 2865): 1.345853

Train time for epoc


Train time for epoch #340 (step 5100): 1.326743

Train time for epoch #341 (step 5115): 1.205271

Train time for epoch #342 (step 5130): 1.250635

Train time for epoch #343 (step 5145): 1.414511

Train time for epoch #344 (step 5160): 1.369097

Train time for epoch #345 (step 5175): 1.215954

Train time for epoch #346 (step 5190): 1.297091

Train time for epoch #347 (step 5205): 1.327746

Train time for epoch #348 (step 5220): 1.225244

Train time for epoch #349 (step 5235): 1.271986

Train time for epoch #350 (step 5250): 1.279423

Train time for epoch #351 (step 5265): 1.399469

Train time for epoch #352 (step 5280): 1.322137

Train time for epoch #353 (step 5295): 1.283153

Train time for epoch #354 (step 5310): 1.277009

Train time for epoch #355 (step 5325): 1.298596

Train time for epoch #356 (step 5340): 1.320534

Train time for epoch #357 (step 5355): 1.374512

Train time for epoch #358 (step 5370): 1.365370

Train time for epoch #359 (step 5385): 1.312992

Train time for epoc


Train time for epoch #508 (step 7620): 1.437156

Train time for epoch #509 (step 7635): 1.347903

Train time for epoch #510 (step 7650): 1.285061

Train time for epoch #511 (step 7665): 1.336951

Train time for epoch #512 (step 7680): 1.360899

Train time for epoch #513 (step 7695): 1.329692

Train time for epoch #514 (step 7710): 1.292437

Train time for epoch #515 (step 7725): 1.333134

Train time for epoch #516 (step 7740): 1.275156

Train time for epoch #517 (step 7755): 1.404454

Train time for epoch #518 (step 7770): 1.280560

Train time for epoch #519 (step 7785): 1.270239

Train time for epoch #520 (step 7800): 1.317948

Train time for epoch #521 (step 7815): 1.200075

Train time for epoch #522 (step 7830): 1.309750

Train time for epoch #523 (step 7845): 1.367495

Train time for epoch #524 (step 7860): 1.328564

Train time for epoch #525 (step 7875): 1.300484

Train time for epoch #526 (step 7890): 1.371091

Train time for epoch #527 (step 7905): 1.273000

Train time for epoc


Train time for epoch #676 (step 10140): 1.300445

Train time for epoch #677 (step 10155): 1.350997

Train time for epoch #678 (step 10170): 1.334382

Train time for epoch #679 (step 10185): 1.497424

Train time for epoch #680 (step 10200): 1.316034

Train time for epoch #681 (step 10215): 1.223638

Train time for epoch #682 (step 10230): 1.292829

Train time for epoch #683 (step 10245): 1.333151

Train time for epoch #684 (step 10260): 1.386769

Train time for epoch #685 (step 10275): 1.293334

Train time for epoch #686 (step 10290): 1.218847

Train time for epoch #687 (step 10305): 1.287326

Train time for epoch #688 (step 10320): 1.377898

Train time for epoch #689 (step 10335): 1.345503

Train time for epoch #690 (step 10350): 1.221390

Train time for epoch #691 (step 10365): 1.382119

Train time for epoch #692 (step 10380): 1.455945

Train time for epoch #693 (step 10395): 1.298222

Train time for epoch #694 (step 10410): 1.330618

Train time for epoch #695 (step 10425): 1.303082


...


Train time for epoch #2775 (step 41625): 1.287942

Train time for epoch #2776 (step 41640): 1.422403

Train time for epoch #2777 (step 41655): 1.354500

Train time for epoch #2778 (step 41670): 1.484840

Train time for epoch #2779 (step 41685): 1.339141

Train time for epoch #2780 (step 41700): 1.202447

Train time for epoch #2781 (step 41715): 1.440637

Train time for epoch #2782 (step 41730): 1.319113

Train time for epoch #2783 (step 41745): 1.378703

Train time for epoch #2784 (step 41760): 1.217823

Train time for epoch #2785 (step 41775): 1.297119

Train time for epoch #2786 (step 41790): 1.200456

Train time for epoch #2787 (step 41805): 1.538841

Train time for epoch #2788 (step 41820): 1.363147

Train time for epoch #2789 (step 41835): 1.309095

Train time for epoch #2790 (step 41850): 1.334667

Train time for epoch #2791 (step 41865): 1.208208

Train time for epoch #2792 (step 41880): 1.454929

Train time for epoch #2793 (step 41895): 1.326519

Train time for epoch #2794 (st


Train time for epoch #2936 (step 44040): 1.502424

Train time for epoch #2937 (step 44055): 1.171797

Train time for epoch #2938 (step 44070): 1.339938

Train time for epoch #2939 (step 44085): 1.386757

Train time for epoch #2940 (step 44100): 1.165267

Train time for epoch #2941 (step 44115): 1.313140

Train time for epoch #2942 (step 44130): 1.306749

Train time for epoch #2943 (step 44145): 1.418357

Train time for epoch #2944 (step 44160): 1.230154

Train time for epoch #2945 (step 44175): 1.320424

Train time for epoch #2946 (step 44190): 1.212249

Train time for epoch #2947 (step 44205): 1.395998

Train time for epoch #2948 (step 44220): 1.315832

Train time for epoch #2949 (step 44235): 1.422251

Train time for epoch #2950 (step 44250): 1.148390

Train time for epoch #2951 (step 44265): 1.328462

Train time for epoch #2952 (step 44280): 1.398199

Train time for epoch #2953 (step 44295): 1.240386

Train time for epoch #2954 (step 44310): 1.404999

Train time for epoch #2955 (st


Train time for epoch #3097 (step 46455): 1.263863

Train time for epoch #3098 (step 46470): 1.442285

Train time for epoch #3099 (step 46485): 1.266080

Train time for epoch #3100 (step 46500): 1.392699

Train time for epoch #3101 (step 46515): 1.204268

Train time for epoch #3102 (step 46530): 1.445385

Train time for epoch #3103 (step 46545): 1.294051

Train time for epoch #3104 (step 46560): 1.356228

Train time for epoch #3105 (step 46575): 1.236476

Train time for epoch #3106 (step 46590): 1.335071

Train time for epoch #3107 (step 46605): 1.289067

Train time for epoch #3108 (step 46620): 1.399911

Train time for epoch #3109 (step 46635): 1.228060

Train time for epoch #3110 (step 46650): 1.418690

Train time for epoch #3111 (step 46665): 1.334280

Train time for epoch #3112 (step 46680): 1.377197

Train time for epoch #3113 (step 46695): 1.425034

Train time for epoch #3114 (step 46710): 1.287472

Train time for epoch #3115 (step 46725): 1.365769

Train time for epoch #3116 (st


Train time for epoch #3258 (step 48870): 1.248141

Train time for epoch #3259 (step 48885): 1.400180

Train time for epoch #3260 (step 48900): 1.413277

Train time for epoch #3261 (step 48915): 1.366643

Train time for epoch #3262 (step 48930): 1.321545

Train time for epoch #3263 (step 48945): 1.229318

Train time for epoch #3264 (step 48960): 1.384948

Train time for epoch #3265 (step 48975): 1.310950

Train time for epoch #3266 (step 48990): 1.342520

Train time for epoch #3267 (step 49005): 1.433230

Train time for epoch #3268 (step 49020): 1.387787

Train time for epoch #3269 (step 49035): 1.235644

Train time for epoch #3270 (step 49050): 1.498680

Train time for epoch #3271 (step 49065): 1.397569

Train time for epoch #3272 (step 49080): 1.432476

Train time for epoch #3273 (step 49095): 1.372991

Train time for epoch #3274 (step 49110): 1.433038

Train time for epoch #3275 (step 49125): 1.202945

Train time for epoch #3276 (step 49140): 1.400377

Train time for epoch #3277 (st


Train time for epoch #3419 (step 51285): 1.491616

Train time for epoch #3420 (step 51300): 1.220023

Train time for epoch #3421 (step 51315): 1.401002

Train time for epoch #3422 (step 51330): 1.219969

Train time for epoch #3423 (step 51345): 1.481430

Train time for epoch #3424 (step 51360): 1.247823

Train time for epoch #3425 (step 51375): 1.357428

Train time for epoch #3426 (step 51390): 1.403854

Train time for epoch #3427 (step 51405): 1.338966

Train time for epoch #3428 (step 51420): 1.261799

Train time for epoch #3429 (step 51435): 1.389725

Train time for epoch #3430 (step 51450): 1.280277

Train time for epoch #3431 (step 51465): 1.471761

Train time for epoch #3432 (step 51480): 1.210070

Train time for epoch #3433 (step 51495): 1.402812

Train time for epoch #3434 (step 51510): 1.240010

Train time for epoch #3435 (step 51525): 1.400057

Train time for epoch #3436 (step 51540): 1.276802

Train time for epoch #3437 (step 51555): 1.396172

Train time for epoch #3438 (st